# Two Sigma Connect: Rental Listing Inquiries

## Summary

This is a machine learning model, built on a Scikit-learn framework, to attempt to solve the [Two Sigma Connect: Rental Listing Inquiries](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries) on [Kaggle](https://www.kaggle.com/). This project was begun by Shaurya Shubham on June 6, 2017,after the competition was completed. In this project he did a peform a feature engineering and stacink of models.

## Things I learnt from this project

* How to make Likelihood feature, which is a Secret Recipe's of Kaggle's GrandMaster.
* How to combine model using Voting-Classifier and Stacking and Essembling.
* How to overcome to frustaion and not to give up spirit.

In [1]:
# Loading Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [62]:
from xgboost import XGBClassifier

C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Loading Datasets
train = pd.read_json('train.json')
test = pd.read_json('test.json')

In [3]:
# Making a copy of datasets
train_ = train
test_ = test

In [4]:
# Seeing the train datasets
print(train.shape)
train.head()

(49352, 15)


,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [5]:
# Let's check the number of columns in train dataset
print("Number of columns in train set:", len(train.columns))
print('=================')
print(train.columns)

Number of columns in train set: 15
Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address'],
      dtype='object')


In [6]:
# Let's check the number of columns in test dataset
print("Number of columns in test set:", len(test.columns))
print('=================')
print(test.columns)

Number of columns in test set: 14
Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'latitude', 'listing_id', 'longitude',
       'manager_id', 'photos', 'price', 'street_address'],
      dtype='object')


In [7]:
# Checking the null values
train.isnull().sum().sort_values(ascending=False).head()

street_address    0
price             0
photos            0
manager_id        0
longitude         0
dtype: int64

In [8]:
# Checking the null values in ascending order to confir null values present or not
test.isnull().sum().sort_values(ascending=False).head()

street_address    0
price             0
photos            0
manager_id        0
longitude         0
dtype: int64

In [9]:
# Let's see some stats of train dataset
train.describe()

,bathrooms,bedrooms,latitude,listing_id,longitude,price
count,49352.00000,49352.000000,49352.000000,4.935200e+04,49352.000000,4.935200e+04
mean,1.21218,1.541640,40.741545,7.024055e+06,-73.955716,3.830174e+03
std,0.50142,1.115018,0.638535,1.262746e+05,1.177912,2.206687e+04
min,0.00000,0.000000,0.000000,6.811957e+06,-118.271000,4.300000e+01
25%,1.00000,1.000000,40.728300,6.915888e+06,-73.991700,2.500000e+03
50%,1.00000,1.000000,40.751800,7.021070e+06,-73.977900,3.150000e+03
75%,1.00000,2.000000,40.774300,7.128733e+06,-73.954800,4.100000e+03
max,10.00000,8.000000,44.883500,7.753784e+06,0.000000,4.490000e+06


In [10]:
# Checking the skewness of the features
train.skew()

bathrooms       2.560563
bedrooms        0.446912
latitude      -63.170742
listing_id      0.251568
longitude      58.964396
price         177.694002
dtype: float64

In [11]:
# Making a list of features that will I use in the prediction
features_to_use = ['bathrooms', 'bedrooms','latitude', 'longitude','price' ]

In [12]:
# Changing categorical feature to numerical
interest_dict = {'low':0, 'medium':1, 'high':2}
train_['interest_level'] = train['interest_level'].map(interest_dict)
train_y = train_['interest_level']

In [13]:
# Correcting the Skewness of target variable
train_['price']=np.log1p(train['price'])
test_['price']=np.log1p(test['price'])

#### Loading the libraries for model generation

In [14]:
from sklearn import model_selection
from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier 

### Performing a basic model on the train datasets to check the resuls

In [15]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = DecisionTreeClassifier()
results = model_selection.cross_val_score(model, train[features_to_use], train_y, cv=kfold ,scoring = 'neg_log_loss')
print(results.mean())

-9.3280614347


In [16]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model2 = RandomForestClassifier(criterion='entropy')
results = model_selection.cross_val_score(model2, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-3.01097300945


In [17]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model3 = RandomForestClassifier(criterion='gini')
results = model_selection.cross_val_score(model3, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-2.98275699


In [18]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model4 = GradientBoostingClassifier()
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.672688713178


In [19]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
eclf = VotingClassifier(estimators=[
    ('model', model), ('model2', model2), ('model3', model3), ('model4', model4)], voting='soft', weights = [1,1,1,3])
results = model_selection.cross_val_score(eclf, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.663612502704


### Let's add some features to improve the performance of the models

In [15]:
train["created"] = pd.to_datetime(train["created"])
train["created_year"] = train["created"].dt.year
train["created_month"] = train["created"].dt.month
train["created_day"] = train["created"].dt.day

test["created"] = pd.to_datetime(test["created"])
test["created_year"] = test["created"].dt.year
test["created_month"] = test["created"].dt.month
test["created_day"] = test["created"].dt.day

In [16]:
features_to_use.append('created_day')
features_to_use.append('created_month')
features_to_use.append('created_year')

In [17]:
train["num_features"] = train["features"].apply(lambda x : len(x))
test["num_features"] = test["features"].apply(lambda x : len(x))

#train2["num_features"] = train["features"].apply(lambda x : len(x))
#test2["num_features"] = test["features"].apply(lambda x : len(x))

features_to_use.append('num_features')

In [24]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.653540154755


In [25]:
train["num_photos"] = train["photos"].apply(len)
test["num_photos"] = test["photos"].apply(len)

#train2["num_photos"] = train["photos"].apply(len)
#test2["num_photos"] = test["photos"].apply(len)

features_to_use.append('num_photos')

##### Checking the results after some feature enginnering

In [26]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.640553310004


#### Let's extract some important feature from text 

In [18]:
from bs4 import BeautifulSoup 
import re
from nltk.corpus import stopwords 

def length_of_description(raw_description):
    
    description_text = BeautifulSoup(raw_description, "html.parser").get_text() # Remove HTML
    letters_only = re.sub("[^a-zA-Z]", " ", description_text) # Remove non letters
    words = letters_only.lower().split() #Convert to lower case, split into individual words
    stops = set(stopwords.words("english")) 
    meaningful_words = [w for w in words if not w in stops] # Remove stop words
    meaningful_words = [w for w in meaningful_words if len(w)>1] # single word if any
    return( len( meaningful_words ))

def clean_description(raw_description):
    
    description_text = BeautifulSoup(raw_description, "html.parser").get_text() # Remove HTML
    letters_only = re.sub("[^a-zA-Z]", " ", description_text) # Remove non letters
    words = letters_only.lower().split() #Convert to lower case, split into individual words
    stops = set(stopwords.words("english")) 
    meaningful_words = [w for w in words if not w in stops] # Remove stop words
    meaningful_words = [w for w in meaningful_words if len(w)>1] # single word if any
    return( " ".join( meaningful_words ))

In [19]:
train['clean_description'] = train['description'].apply(lambda x : clean_description(x))
test['clean_description'] = test['description'].apply(lambda x : clean_description(x))

#train2['clean_description'] = train['description'].apply(lambda x : clean_description(x))
#test2['clean_description'] = test['description'].apply(lambda x : clean_description(x))

C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [20]:
# Let's add length of description as a feature
train['description_length'] = train['description'].apply(lambda x : length_of_description(x))
test['description_length'] = test['description'].apply(lambda x : length_of_description(x))

#train2['description_length'] = train['description'].apply(lambda x : length_of_description(x))
#test2['description_length'] = test['description'].apply(lambda x : length_of_description(x))

features_to_use.append('description_length')

C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [30]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.639786153295


#### Now add some more features to further increase the performance

In [21]:
train["price_t"] =train["price"]/train["bedrooms"]
test["price_t"] = test["price"]/test["bedrooms"] 

train['room_sum'] = train['bedrooms']  + train['bathrooms']
test['room_sum'] = test['bedrooms']  + test['bathrooms']

train['price_per_room'] = train['price']/train['room_sum']
test['price_per_room'] = test['price']/test['room_sum']

train['price_per_room'] = train['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
test['price_per_room'] = test['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
train['price_t'] = train['price_t'].apply(lambda x: 0 if x==np.inf else x)
test['price_t'] = test['price_t'].apply(lambda x: 0 if x==np.inf else x)

In [ ]:
train2["price_t"] =train["price"]/train["bedrooms"]
test2["price_t"] = test["price"]/test["bedrooms"] 

train2['room_sum'] = train['bedrooms']  + train['bathrooms']
test2['room_sum'] = test['bedrooms']  + test['bathrooms']

train2['price_per_room'] = train['price']/train['room_sum']
test2['price_per_room'] = test['price']/test['room_sum']

In [23]:
train['price_per_room'] = train['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
test['price_per_room'] = test['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
train['price_t'] = train['price_t'].apply(lambda x: 0 if x==np.inf else x)
test['price_t'] = test['price_t'].apply(lambda x: 0 if x==np.inf else x)

#train2['price_per_room'] = train['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
#test2['price_per_room'] = test['price_per_room'].apply(lambda x: 0 if x==np.inf else x)
#train2['price_t'] = train['price_t'].apply(lambda x: 0 if x==np.inf else x)
#test2['price_t'] = test['price_t'].apply(lambda x: 0 if x==np.inf else x)

In [24]:
features_to_use.append('price_t')
features_to_use.append('room_sum')
features_to_use.append('price_per_room')

In [33]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.638719296049


In [ ]:
train2['price']=np.log1p(train['price'])
test2['price']=np.log1p(test['price'])

train2['price_per_room']=np.log1p(train['price_per_room'])
test2['price_per_room']=np.log1p(test['price_per_room'])

train2['price_t']=np.log1p(train['price_t'])
test2['price_t']=np.log1p(test['price_t'])

In [ ]:
results = model_selection.cross_val_score(model4, train2[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

In [25]:
kfold2 = model_selection.KFold( n_splits =10,  random_state= 12)
kfold3 = model_selection.KFold( n_splits =10,  random_state= 12)

### Creating a Likelihood features(it's a magic feature) using manager_id and Building_id

In [26]:
train2 = train[['manager_id','interest_level']]

In [27]:
train2['manager_skill'] =np.nan

for train_idx, test_idx in kfold3.split(train2):
    trainX, testX = train2.iloc[train_idx], train2.iloc[test_idx]
    trainy, testy = train_y.iloc[train_idx], train_y.iloc[test_idx]
    

    for train_idx2, test_idx2 in kfold3.split(trainX):
        trainX2, testX2 = trainX.iloc[train_idx2], trainX.iloc[test_idx2]
        trainy2, testy2 = trainy.iloc[train_idx2], trainy.iloc[test_idx2]
        
        trainX2=trainX2.drop('manager_skill',axis=1)
        
        temp = pd.concat([trainX2['manager_id'], pd.get_dummies(trainX2['interest_level'])],axis =1).groupby('manager_id').mean()
        temp.columns = ['low_frac','medium_frac', 'high_frac']
        #print(temp.head())
        temp['count'] = trainX2.groupby('manager_id').count()

        
        temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']
        mean_skill = temp.loc[temp['count'] >= 5, 'manager_skill'].mean() # setting a thresold
        temp.loc[temp['count'] < 2, 'manager_skill'] = mean_skill # if count is less than thresold impute the mean

        testX2=testX2.drop('manager_skill',axis=1)
        testX2 = testX2.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
        testX2['manager_skill'].fillna(mean_skill, inplace= True) # if manager_id is not found fill it by mean value
      
        trainX.iloc[list(test_idx2), trainX.columns.get_loc('manager_skill')] = list(testX2['manager_skill'])


    testX=testX.drop('manager_skill',axis=1)
    trainX=trainX.drop('interest_level',axis=1)
    
    temp2 = trainX.groupby('manager_id').mean()
    testX = testX.merge(temp2.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
    testX['manager_skill'].fillna(testX['manager_skill'].dropna().mean(), inplace= True)
    
    train2.iloc[list(test_idx), train2.columns.get_loc('manager_skill')] = list(testX['manager_skill'])

C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [28]:
train['manager_skill'] = train2['manager_skill']

In [29]:
features_to_use.append('manager_skill')

In [39]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.588127625952


In [123]:
test_ = pd.read_json('test.json')

In [30]:
test2= test_[['manager_id']]

In [31]:
temp3 = train2.groupby('manager_id').mean()
#print((temp3.head()))
temp3 = temp3.drop('interest_level',axis=1)
#print((temp3.head()))
test2 = test2.merge(temp3.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
print(test2.head())
test2['manager_skill'].fillna(test2['manager_skill'].dropna().mean(), inplace= True)

                         manager_id  manager_skill
0  b1b1852c416d78d7765d746cb1b8921f       0.800746
1  d0b5648017832b2427eeb9956d966a14       0.000000
2  9ca6f3baa475c37a3b3521a394d65467       0.390975
3  0b9d5db96db8472d7aeb67c67338c4d2       1.090504
4  b5eda0eb31b042ce2124fd9e9fcfce2f       0.482316


In [32]:
test2.head()

,manager_id,manager_skill
0,b1b1852c416d78d7765d746cb1b8921f,0.800746
1,d0b5648017832b2427eeb9956d966a14,0.000000
2,9ca6f3baa475c37a3b3521a394d65467,0.390975
3,0b9d5db96db8472d7aeb67c67338c4d2,1.090504
4,b5eda0eb31b042ce2124fd9e9fcfce2f,0.482316


In [33]:
test_[['manager_id']].head()

,manager_id
0,b1b1852c416d78d7765d746cb1b8921f
1,d0b5648017832b2427eeb9956d966a14
100,9ca6f3baa475c37a3b3521a394d65467
1000,0b9d5db96db8472d7aeb67c67338c4d2
100000,b5eda0eb31b042ce2124fd9e9fcfce2f


In [44]:
test.drop('manager_skill', axis=1, inplace=True)

ValueError: labels ['manager_skill'] not contained in axis

In [34]:
test= test.reset_index(drop = True)

In [35]:
test['manager_skill'] = test2['manager_skill']

In [36]:
test['manager_skill'].isnull().sum()

0

In [37]:
train3 = train[['building_id','interest_level']]

In [38]:
train3['building_value'] =np.nan

for train_idx, test_idx in kfold3.split(train3):
    trainX, testX = train3.iloc[train_idx], train3.iloc[test_idx]
    trainy, testy = train_y.iloc[train_idx], train_y.iloc[test_idx]
    

    for train_idx2, test_idx2 in kfold3.split(trainX):
        trainX2, testX2 = trainX.iloc[train_idx2], trainX.iloc[test_idx2]
        trainy2, testy2 = trainy.iloc[train_idx2], trainy.iloc[test_idx2]

        
        trainX2=trainX2.drop('building_value',axis=1)
        
        temp = pd.concat([trainX2['building_id'], pd.get_dummies(trainX2['interest_level'])],axis =1).groupby('building_id').mean()
        temp.columns = ['low_frac','medium_frac', 'high_frac']
        #print(temp.head())
        temp['count'] = trainX2.groupby('building_id').count()

        
        temp['building_value'] = temp['high_frac']*2 + temp['medium_frac']
        mean_value = temp.loc[temp['count'] >= 5, 'building_value'].mean() # setting a thresold
        temp.loc[temp['count'] < 2, 'building_value'] = mean_value # if count is less than thresold impute the mean

        testX2=testX2.drop('building_value',axis=1)
        testX2 = testX2.merge(temp.reset_index(),how='left', left_on='building_id', right_on='building_id')
        testX2['building_value'].fillna(mean_skill, inplace= True) # if manager_id is not found fill it by mean value
      
        trainX.iloc[list(test_idx2), trainX.columns.get_loc('building_value')] = list(testX2['building_value'])


    testX=testX.drop('building_value',axis=1)
    trainX=trainX.drop('interest_level',axis=1)
    
    temp2 = trainX.groupby('building_id').mean()
    testX = testX.merge(temp2.reset_index(),how='left', left_on='building_id', right_on='building_id')
    testX['building_value'].fillna(testX['building_value'].dropna().mean(), inplace= True)
    
    train3.iloc[list(test_idx), train3.columns.get_loc('building_value')] = list(testX['building_value'])

C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Shaurya\Anaconda2\envs\tensorflow-gpu\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [39]:
train['building_value'] = train3['building_value']

In [40]:
features_to_use.append('building_value')

In [52]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.577978172591


In [41]:
test3= test[['building_id']]

In [42]:
temp4 = train3.groupby('building_id').mean()
test3 = test3.merge(temp4.reset_index(),how='left', left_on='building_id', right_on='building_id')
test3['building_value'].fillna(testX['building_value'].dropna().mean(), inplace= True)

In [43]:
test['building_value'] = test3['building_value']

In [44]:
test['building_value'].isnull().sum()

0

In [45]:
from sklearn import preprocessing

categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train[f].dtype=='object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train[f].values) + list(test[f].values))
            train[f] = lbl.transform(train[f].values)
            test[f] = lbl.transform(test[f].values)
            features_to_use.append(f)

#### Let's Check the performance after using magic features

In [58]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.576855551283


#### creating text count as a feature(But it doesn't improve the performance)

In [86]:
from sklearn.feature_extraction.text import  CountVectorizer

tfidf = CountVectorizer(stop_words='english', max_features=200)
train_sparse = tfidf.fit_transform(train["clean_description"])
test_sparse = tfidf.transform(test["clean_description"])

In [87]:
from scipy import sparse

train_X = sparse.hstack([train[features_to_use], train_sparse]).tocsr()
#test_X = sparse.hstack([test[features_to_use], test_sparse]).tocsr()

In [88]:
train_X = train_X.toarray()
#test_X = test_X.toarray()

In [89]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.576854680343


In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [92]:
tfidfdesc=TfidfVectorizer(min_df=10, max_features=50)
train_sparsed = tfidfdesc.fit_transform(train["clean_description"])
#test_sparsed = tfidfdesc.transform(test["clean_description"])

In [93]:
train_X2 = sparse.hstack([train[features_to_use], train_sparsed]).tocsr()
#test_X2 = sparse.hstack([test2[features_to_use], test_sparsed]).tocsr()

In [94]:
train_X2 = train_X2.toarray()
#test_X2 = test_X2.toarray()

In [95]:
results = model_selection.cross_val_score(model4, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.576860161965


In [96]:
train_X3 = sparse.hstack([train[features_to_use], train_sparse,train_sparsed]).tocsr()#\
#test_X3 = sparse.hstack([test2[features_to_use], test_sparse,test_sparsed]).tocsr()

In [97]:
train_X3 = train_X3.toarray()
#test_X3 = test_X3.toarray()

In [98]:
results = model_selection.cross_val_score(model4,train_X, train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.578075546664


#### Using different models for the predictions and mentioning the Log-loss of each model on Leader Board

In [64]:
random_state = 54321
rf1 = RandomForestClassifier(criterion='entropy',  n_jobs = -1,  random_state=random_state)
rf2= RandomForestClassifier( criterion='gini',  n_jobs = -1, random_state=random_state)
gbc = GradientBoostingClassifier(random_state=random_state)

eclf = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('gbc', gbc)], voting='soft')
eclf.fit(train[features_to_use], train_y)
y_pred_test =eclf.predict_proba(test[features_to_use])          

In [46]:
labels2idx = {'low': 0, 'medium': 1, 'high': 2}
labels2idx

{'high': 2, 'low': 0, 'medium': 1}

In [165]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test[:, labels2idx[label]]
sub.to_csv("submission3_eclf.csv", index=False)

### LB : 0.58986

In [166]:
eclf2 = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('gbc', gbc)], voting='soft', weights = [1,1,3])
eclf2.fit(train[features_to_use], train_y)
y_pred_test2 =eclf2.predict_proba(test[features_to_use])  

In [167]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test2[:, labels2idx[label]]
sub.to_csv("submission3_eclf2.csv", index=False)

### LB : 0.58005

------------------------------------------------------

In [79]:
from sklearn.ensemble import AdaBoostClassifier , ExtraTreesClassifier

In [60]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model5 = AdaBoostClassifier()
results = model_selection.cross_val_score(model5, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-1.06498336496


In [61]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model5 = ExtraTreesClassifier()
results = model_selection.cross_val_score(model5, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-1.50425540146


In [63]:
random_state = 54321
dt = DecisionTreeClassifier(min_samples_leaf= 500)
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1)
rf22= RandomForestClassifier( criterion='gini',n_estimators=300, max_features= 'sqrt', n_jobs = -1)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf13 = VotingClassifier(estimators=[('dt',dt),('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3,3])
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
results = model_selection.cross_val_score(eclf13, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())
#eclf12.fit(train[features_to_use], train_y)
#y_pred_test12 =eclf12.predict_proba(test[features_to_use])

-0.555059922796


In [64]:
eclf13.fit(train[features_to_use], train_y)
y_pred_test13 =eclf13.predict_proba(test[features_to_use])

In [65]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test13[:, labels2idx[label]]
sub.to_csv("submission3_eclf13.csv", index=False)

### LB : 0.56537

In [66]:
adc = AdaBoostClassifier(random_state=random_state)
extc = ExtraTreesClassifier(random_state=random_state)

eclf3 = VotingClassifier(estimators=[('rf1', rf1), ('rf2', rf2), ('gbc', gbc), ('adc', adc), ('extc', extc)], 
                         voting='soft', weights = [1,1,4,1,1])

eclf3.fit(train[features_to_use], train_y)
y_pred_test3 =eclf.predict_proba(test[features_to_use])

In [67]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test3[:, labels2idx[label]]
sub.to_csv("submission3_eclf3.csv", index=False)

-----------------------------------------------------------

###### Hyperparameter tuning

In [68]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [91]:
seed = 7
kfold = model_selection.KFold(n_splits=10,  random_state=seed)
model_a = RandomForestClassifier(criterion='gini' , n_estimators=300, max_features= 'sqrt')
results = model_selection.cross_val_score(model_a, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.5719469538


In [93]:
seed = 7
kfold = model_selection.KFold(n_splits=10,  random_state=seed)
model_b = RandomForestClassifier(criterion='entropy' , n_estimators=400, max_features= 'sqrt')
results = model_selection.cross_val_score(model_b, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.568437783474


In [88]:
seed = 7
kfold = model_selection.KFold(n_splits=10,  random_state=seed)
model_c = GradientBoostingClassifier(n_estimators=500)
results = model_selection.cross_val_score(model_c, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.555977146749


In [89]:
seed = 7
kfold = model_selection.KFold(n_splits=10,  random_state=seed)
model_d = GradientBoostingClassifier( n_estimators=450)
results = model_selection.cross_val_score(model_d, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.556619530533


In [265]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model_xgb = XGBClassifier(n_estimators=500)
results = model_selection.cross_val_score(model_xgb, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.555350287445


In [292]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model5 = DecisionTreeClassifier(min_samples_leaf= 500)
results = model_selection.cross_val_score(model5, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.62488310256


--------------------------------

In [94]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)

eclf4 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2)], voting='soft')
eclf4.fit(train[features_to_use], train_y)
y_pred_test4 =eclf4.predict_proba(test[features_to_use])  

In [95]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test4[:, labels2idx[label]]
sub.to_csv("submission3_eclf4.csv", index=False)

### LB : 0.56684

In [268]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf6 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft')
eclf6.fit(train[features_to_use], train_y)
y_pred_test6 =eclf6.predict_proba(test[features_to_use]) 

In [269]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test6[:, labels2idx[label]]
sub.to_csv("submission3_eclf6.csv", index=False)

### LB : 0.56290

In [96]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)

eclf5 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2)], voting='soft', weights = [1,1,3])
eclf5.fit(train[features_to_use], train_y)
y_pred_test5 =eclf5.predict_proba(test[features_to_use])  

In [97]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test5[:, labels2idx[label]]
sub.to_csv("submission3_eclf5.csv", index=False)

### LB : 0.56210

In [270]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf7 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,3,3])
eclf7.fit(train[features_to_use], train_y)
y_pred_test =eclf7.predict_proba(test[features_to_use])

In [271]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test[:, labels2idx[label]]
sub.to_csv("submission3_eclf7.csv", index=False)

### LB : 0.56079

In [272]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf8 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3])
eclf8.fit(train[features_to_use], train_y)
y_pred_test8 =eclf8.predict_proba(test[features_to_use])

In [273]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test8[:, labels2idx[label]]
sub.to_csv("submission3_eclf8.csv", index=False)

### LB : 0.56107

In [274]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy', max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini', max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf9 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3])
eclf9.fit(train[features_to_use], train_y)
y_pred_test9 =eclf9.predict_proba(test[features_to_use])

In [275]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test9[:, labels2idx[label]]
sub.to_csv("submission3_eclf9.csv", index=False)

### LB : 0.56541

In [276]:
random_state = 54321
dt = DecisionTreeClassifier(random_state=random_state)
rf12 = RandomForestClassifier(criterion='entropy', max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini', max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf10 = VotingClassifier(estimators=[('dt',dt),('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,1,3])
eclf10.fit(train[features_to_use], train_y)
y_pred_test10 =eclf10.predict_proba(test[features_to_use])

In [277]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test10[:, labels2idx[label]]
sub.to_csv("submission3_eclf10.csv", index=False)

### LB : 0.57536

In [278]:
random_state = 54321
dt = DecisionTreeClassifier(random_state=random_state)
rf12 = RandomForestClassifier(criterion='entropy', max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini', max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf11 = VotingClassifier(estimators=[('dt',dt),('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3,3])
eclf11.fit(train[features_to_use], train_y)
y_pred_test11 =eclf11.predict_proba(test[features_to_use])

In [279]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test11[:, labels2idx[label]]
sub.to_csv("submission3_eclf11.csv", index=False)

### LB : 0.56934

In [280]:
random_state = 54321
dt = DecisionTreeClassifier(random_state=random_state)
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf12 = VotingClassifier(estimators=[('dt',dt),('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3,3])
eclf12.fit(train[features_to_use], train_y)
y_pred_test12 =eclf12.predict_proba(test[features_to_use])

In [281]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test12[:, labels2idx[label]]
sub.to_csv("submission3_eclf12.csv", index=False)

### LB : 0.56709

-------------------------------

##### Stacking and ensembling

In [148]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 10 # set folds for out-of-fold prediction
kf5 = model_selection.KFold( n_splits= NFOLDS, random_state=SEED)

In [149]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    

In [150]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf1_params = {
    'n_jobs': -1,
    'n_estimators': 400,
    "criterion":'entropy',
    'max_features' : 'sqrt'
}

# Random Forest parameters
rf2_params = {
    'n_jobs': -1,
    'n_estimators':300,
    'max_features' : 'sqrt',
    'criterion' :'gini'
}

# AdaBoost parameters
gb_params = {
    'n_estimators': 450
}



In [193]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf5.split(train[features_to_use])):
        x_tr = x_train.iloc[train_index]
        y_tr = y_train.iloc[train_index]
        x_te = x_train.iloc[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [194]:
rf_basemodel_1 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf1_params)
rf_basemodel_2 = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf2_params)

In [195]:
x_train = train[features_to_use] # Creates an array of the train data
x_test = test[features_to_use] # Creats an array of the test data
y_train =train_y

In [196]:
rf1_oof_train, rf1_oof_test = get_oof(rf_basemodel_1,x_train, y_train, x_test) # Random Forest1
rf2_oof_train, rf2_oof_test = get_oof(rf_basemodel_2,x_train, y_train, x_test) # Random Forest2

In [197]:
rf1_oof_train.shape

(49352, 1)

In [206]:
rf1_oof_test.shape

(74659, 1)

In [207]:
rf1_feature = rf_basemodel_1.feature_importances(x_train,y_train)
rf2_feature = rf_basemodel_2.feature_importances(x_train,y_train)

[ 0.0057872   0.00785919  0.05311362  0.05460172  0.08334348  0.05313182
  0.01878873  0.          0.039833    0.03920931  0.05682042  0.05209093
  0.00957201  0.05594686  0.15865577  0.09835107  0.05227153  0.04881335
  0.05915259  0.05265739]
[ 0.00570351  0.00803992  0.05320849  0.05439168  0.08150379  0.05546506
  0.01884059  0.          0.04029595  0.03825217  0.05708406  0.0505135
  0.00968644  0.05630707  0.15637819  0.09947278  0.05313319  0.04923369
  0.05964939  0.05284051]


In [215]:
rf1_features = [ 0.0057872,   0.00785919,  0.05311362,  0.05460172,  0.08334348,  0.05313182,
  0.01878873,  0.0,          0.039833,    0.03920931,  0.05682042,  0.05209093,
  0.00957201,  0.05594686,  0.15865577,  0.09835107,  0.05227153,  0.04881335,
  0.05915259,  0.05265739]

In [216]:
rf2_features = [ 0.00570351,  0.00803992,  0.05320849,  0.05439168,  0.08150379,  0.05546506,
  0.01884059,  0.0,          0.04029595,  0.03825217,  0.05708406,  0.0505135,
  0.00968644,  0.05630707,  0.15637819,  0.09947278,  0.05313319,  0.04923369,
  0.05964939,  0.05284051]

In [219]:

# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': features_to_use,
     'Random Forest_1 feature importances': rf1_features,
    'Random Forest_2 feature importances': rf2_features                            
    })

In [220]:
feature_dataframe

,Random Forest_1 feature importances,Random Forest_2 feature importances,features
0,0.005787,0.005704,bathrooms
1,0.007859,0.008040,bedrooms
2,0.053114,0.053208,latitude
3,0.054602,0.054392,longitude
4,0.083343,0.081504,price
5,0.053132,0.055465,created_day
6,0.018789,0.018841,created_month
7,0.000000,0.000000,created_year
8,0.039833,0.040296,num_features
9,0.039209,0.038252,num_photos


In [227]:

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.sort_values(by='mean' ,ascending=False)

,Random Forest_1 feature importances,Random Forest_2 feature importances,features,mean
14,0.158656,0.156378,manager_skill,0.157517
15,0.098351,0.099473,building_value,0.098912
4,0.083343,0.081504,price,0.082424
18,0.059153,0.059649,building_id,0.059401
10,0.056820,0.057084,description_length,0.056952
13,0.055947,0.056307,price_per_room,0.056127
3,0.054602,0.054392,longitude,0.054497
5,0.053132,0.055465,created_day,0.054298
2,0.053114,0.053208,latitude,0.053161
19,0.052657,0.052841,street_address,0.052749


In [230]:
base_predictions_train = pd.DataFrame( {'RandomForest1': rf1_oof_train.ravel(),
     'RandomForest2': rf2_oof_train.ravel()
    })
base_predictions_train.head()

,RandomForest1,RandomForest2
0,0.0,0.0
1,0.0,0.0
2,1.0,1.0
3,0.0,0.0
4,0.0,0.0


In [231]:
x_train = np.concatenate((  rf1_oof_train, rf2_oof_train ), axis=1)
x_test = np.concatenate((  rf1_oof_test, rf2_oof_test ), axis=1)

In [233]:
x_train.shape

(49352, 2)

In [234]:
x_test.shape

(74659, 2)

In [246]:
gbc_stack = GradientBoostingClassifier( random_state=random_state)
gbc_stack.fit(x_train, y_train)
predictions = gbc_stack.predict_proba(x_test)

In [247]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = predictions[:, labels2idx[label]]
sub.to_csv("submission3_stack1.csv", index=False)

In [245]:
predictions

array([[ 0.28288313,  0.51214877,  0.20496811],
       [ 0.81696696,  0.15856797,  0.02446507],
       [ 0.81696696,  0.15856797,  0.02446507],
       ..., 
       [ 0.81696696,  0.15856797,  0.02446507],
       [ 0.28288313,  0.51214877,  0.20496811],
       [ 0.81696696,  0.15856797,  0.02446507]])

#### Adding Some more features to see the results

In [53]:
import Levenshtein

In [54]:
train_ = pd.read_json('train.json')
test_ = pd.read_json('test.json')

In [55]:
train_['display_address'][10].lower()

'metropolitan avenue'

In [56]:
train_['street_address'] [10]

'792 Metropolitan Avenue'

In [57]:
Levenshtein.ratio(train_['display_address'][10].lower(), train_['street_address'] [10].lower())

0.9047619047619048

In [58]:
def get_leven_ratio_row(row):
    return Levenshtein.ratio(row.display_address.lower(), row.street_address.lower())

def get_leven_ratio(df):
    return df.apply(get_leven_ratio_row, axis=1)



In [66]:
train['address_ratio'] =get_leven_ratio(train_)

In [71]:
test['address_ratio'] =get_leven_ratio(test_).reset_index(drop=True)

In [72]:
test['address_ratio'].isnull().sum()

0

In [73]:
features_to_use.append('address_ratio')

In [74]:
random_state = 54321
dt = DecisionTreeClassifier(min_samples_leaf= 500)
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1)
rf22= RandomForestClassifier( criterion='gini',n_estimators=300, max_features= 'sqrt', n_jobs = -1)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf14 = VotingClassifier(estimators=[('dt',dt),('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,1,3,3])
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
results = model_selection.cross_val_score(eclf14, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.553786014577


In [75]:
eclf14.fit(train[features_to_use], train_y)
y_pred_test14 =eclf14.predict_proba(test[features_to_use])

In [76]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test14[:, labels2idx[label]]
sub.to_csv("submission3_eclf14.csv", index=False)

### LB : 0.56456

In [77]:
random_state = 54321
rf12 = RandomForestClassifier(criterion='entropy',n_estimators=400, max_features= 'sqrt',  n_jobs = -1,  random_state=random_state)
rf22= RandomForestClassifier( criterion='gini',  n_estimators=300, max_features= 'sqrt', n_jobs = -1, random_state=random_state)
gbc2 = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
xgb = XGBClassifier(n_estimators=500)

eclf15 = VotingClassifier(estimators=[('rf12', rf12), ('rf22', rf22), ('gbc2', gbc2), ('xgb', xgb)], voting='soft', weights = [1,1,3,3])
eclf15.fit(train[features_to_use], train_y)
y_pred_test15 =eclf15.predict_proba(test[features_to_use])

In [78]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test15[:, labels2idx[label]]
sub.to_csv("submission3_eclf15.csv", index=False)

### LB : 0.56165

In [80]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model8 = AdaBoostClassifier()
results = model_selection.cross_val_score(model8, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-1.06586045927


In [101]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model8 = AdaBoostClassifier(n_estimators= 100, learning_rate= 0.001)
results = model_selection.cross_val_score(model8, train[features_to_use], train_y, cv=kfold, scoring = 'neg_log_loss')
print(results.mean())

-0.705734073619


----------------

In [102]:
gbc_model = GradientBoostingClassifier(n_estimators=450, random_state=random_state)
gbc_model.fit(train[features_to_use], train_y)
y_pred_test16 = gbc_model.predict_proba(test[features_to_use])

In [103]:
sub = pd.DataFrame()
sub["listing_id"] = test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y_pred_test16[:, labels2idx[label]]
sub.to_csv("submission3_gbcmodel.csv", index=False)

### LB : 0.56712